Rodrigo Hernandez  
Muzamil Syed  
Mayra Gamboa  
CS178  
<h3 align='center'>Group Project</b>

<i><h2 align='center'>Introduction to Our Technique</h2></i>  
The approach we will be taking will have to do with going into detail with Ensembles. Instead of having an ensemble of only one type of learner, we decided to have an ensemble with three different types of learners. These learners will be decision trees, K-Nearest-Neighbours, and support vector machines.  

We will each choose our own type of learner and perform our own experiments to try and maximize the accuracy of our learner's predictions. After we have chosen our optimal learners, we will combine them into an ensemble of size 24 (8 learners of each type) and predict the test data using this new ensemble.  

We will use existing packages provided by sklearn library. We will use the methods provided by the library and explore additional techniques to supplement these methods to vary the complexities of the models that result.

In [ ]:
#Report Format: (Maybe me we could have some kind of title page added to final pdf with our names) 

In [2]:
##########Libraries Used Throughout The Code:##########

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import mltools as ml
import mltools.dtree as dtree
import mltools.logistic2 as lcs2
import sklearn
from sklearn import svm
from sklearn import preprocessing
%matplotlib inline
#Imported Library for Neural Network 
#Imported Library for Third Learner 

In [3]:
##########Imported Data:##########

'''
We will use the provided Class Kaggle Data in our class Kaggle Competition: CS178 Project 2016 

https://inclass.kaggle.com/c/cs178-project-2016
'''  

# Get Kaggle training data
X = np.genfromtxt("data/kaggle.X1.train.txt",delimiter=",")
Y = np.genfromtxt("data/kaggle.Y.train.txt",delimiter=",")

# also load features of the test data (to be predicted)
Xe1 = np.genfromtxt("data/kaggle.X1.test.txt",delimiter=",")

perSplit = 0.8 # Percent at which to split the training data
               # (e.g 0.8 = 80/20 split)

Xtr,Xte,Ytr,Yte = ml.splitData(X,Y,0.8)

print(X.shape)

(60000, 91)


In [21]:
##########Initialization of the Ensemble:##########

'''
We will start with an Ensemble of size 25...
''' 

# Ensemble Variables
size = 25  # the amount of learners in the ensemble
features = 55 # the number of features to select from when bagging

# Create the ensemble
ensemble = [None] * size


(60000, 91)


Since the Kaggle data has 91 dimensions, one of the type of learners we chose was a support vector machine, as they perform well in high dimensionality. First, we tested different types of kernels to find which one performed the best. It is obvious that linear kernel was not going to estimate the data correctly, so a linear kernal was not tested.

In [7]:
##########Code for the Support Vector Machines:##########

# First we scale the data for the SVM
XiTe = preprocessing.scale(Xte)
XiTr = preprocessing.scale(Xtr)

# Test accuracy of each type of SVM
kernels = ['rbf','sigmoid','poly']

for k in kernels:
    clf = svm.SVR(kernel=k)
    clf.fit(XiTr[:1000,],Ytr[:1000])
    
    YhatTrain = clf.predict(Xtr)
    YhatTest = clf.predict(Xte)

    MSEtrain = np.mean((Ytr - YhatTrain)**2)
    MSEtest = np.mean((Yte - YhatTest)**2)

    print("SVM with {} as kernal".format(k))
    print("\tMSE of training data: " + str(MSEtrain))
    print("\tMSE of test data: " + str(MSEtest))

SVM with rbf as kernal
	MSE of training data: 0.690871165546
	MSE of test data: 0.718522580681
SVM with sigmoid as kernal
	MSE of training data: 0.780008793301
	MSE of test data: 0.814328865355
SVM with poly as kernal
	MSE of training data: 8.00810410361e+26
	MSE of test data: 8.12192003453e+26


Based on the data, the RBF kernel had the best performance. Now, since SVM can take a long time with large amounts of data, we will see how long an SVM with a RBF kernel takes with different subsets of data and it's performance on said data.

In [28]:
import time

clf = svm.SVR(kernel='rbf')
Xi = preprocessing.scale(X)

for i in [5000,10000,20000,40000,60000]:
    print("Data for {} data points".format(i))
    t0 = time.time()
    clf.fit(Xi[:i,],Y[:i])
    
    print("\tTraining: {:.2f} seconds".format(time.time()-t0))
    
    t0 = time.time()
    Yhat = clf.predict(X[:i,])
    print("\tPredicting: {:.2f} seconds".format(time.time()-t0))
    
    MSE = np.mean((Y[:i] - Yhat)**2)

    print("\tMSE of data: {:.2f}".format(MSE))
    
    
    

Data for 5000 data points
	Training: 3.24 seconds
	Predicting: 1.79 seconds
	MSE of data: 0.71
Data for 10000 data points
	Training: 12.60 seconds
	Predicting: 6.96 seconds
	MSE of data: 0.70
Data for 20000 data points
	Training: 180.33 seconds
	Predicting: 28.04 seconds
	MSE of data: 0.70
Data for 40000 data points
	Training: 890.83 seconds
	Predicting: 112.21 seconds
	MSE of data: 0.69
Data for 60000 data points
	Training: 3303.78 seconds
	Predicting: 245.58 seconds
	MSE of data: 0.70


In [ ]:
temp = [None] * 8

# Train model
for i in range(8):
    Xi,Yi = ml.utils.bootstrapData(Xtr,Ytr)
    temp[i] = svm.SVR(kernel='rbf')
    temp[i].fit(XiTr,Ytr)

In [ ]:
# Now predict
mTest = Xte.shape[0]
predict = np.zeros((mTest,8))
for learner in temp:
    predict[:,i] = temp[i].predict(Xte)
    
predict = np.mean(predict,axis=1)

print("MSE of ensemble of length 8: {}".format(np.mean((Yte-predict)**2)))

In [ ]:
##########Code for the KNN Learner:##########
from sklearn.neighbors import KNeighborsRegressor  #Imported Library for KNN Regressor 

'''
KNN Learner INFO... 
-describe problem you use chose and methods to address it
-how did you train the models?
-how you selected any parameters each model/method requires 
-how they performed on test data
-consider table of performance of different approaches or plots of perofrmance used to perform model selection
''' 

'''
Key Points:
-Explore some aspect of prediction that we have not already done in depth 
-Identify a paper that proposes a method you think could be helpful 
-Use stacking/information from your leaderboard performance to try and improve your prediction quality 
-To explore approach: explore method from class fully enough to understand how changes might affect its performance,
verify your findings make sense, and then use your findings to optimize performance
-In Your Report: describe why you decided to explore this aspect, what you expected to find, and how your findings
matched/ didnt match your expectations 
-Beware of the positive/negative aspects of the learners we discussed ie Nearest Neighbor methods can be powerful but can
also be slow for large data sets...perhaps you can reduce the data in some way without sacrificing performance (bootstrap
aggregation)
-Linear methods may be fast but do not provide enough model complexity to provide a good fit so you may been to try and 
generate better features 
'''


In [ ]:
##########Code for the Decision Tree:##########
#Imported Library for Decision Trees   

'''
Decision Tree Learner INFO... 
'''

# Decision Tree Variables
depth = 20 # the maxth depth of the decision tree
nodes = 8 # the minimum number of data to split node

In [ ]:
##########Code to store each learner in the ensemble:##########

'''
How should we store the learners we come up with? Since right now we have an ensemble of size 25 we could maybe 
create a list for each of our learners and bring them here then index each list of learners, train it and then store 
it into the ensemble before making all the predictions. What do you guys think?  
'''
# Create learners and add to ensemble

for i in range(size):
    
    #for learners in each list 
    
        #get KNN learner 
        #train KNN learner 
        #ensemble[i] = KNN Learner 
        
        #get NN learner 
        #train NN learner 
        #ensemble[i] = NN Learner 

        #get DT learner 
        #train DT learner 
        #ensemble[i] = DT Learner 
        
    #dt = dtree.treeRegress()
    #Xi,Yi = ml.utils.bootstrapData(Xtr,Ytr)
    #dt.train(Xi,Yi,maxdepth=depth,nFeatures=features,minParent=nodes)
    
    #ensemble[i] = dt

In [ ]:
##########Code to output the predictions and evaluate them on kaggle:##########

# Test correctness of ensemble through MSE
mTest = Xte.shape[0] # Acquire the shape of the test data
Yhat = np.zeros((mTest,num))
MSE = 0

for i in range(size):
    Yhat[:,i] = ensemble[i].predict(Xte).reshape(mTest)
    
    Yhat = np.mean(Yhat,axis=1)
    
    MSE = np.mean((Yte - Yhat.reshape(Yte.shape))**2,axis=0)
    
print(MSE)

'''
Note:
-Should not try to upload every possible model with every possible parameter setting 
-Use validation data, or cross-validation to assess which models are worth uploading, and just use the uploads
to verify performance. 
'''
#Ye = learner.predict( Xeval ); # make predictions
# Note: be sure Ye is a flat vector, shape (m,)
# otherwise, reshape it using e.g.
# Ye = Ye.ravel()
# or change the indexing in the code below:
fh = open('predictions.csv','w') # open file for upload
fh.write('ID,Prediction\n') # output header line
for i,yi in enumerate(Ye):
fh.write('{},{}\n'.format(i+1,yi)) # output each prediction
fh.close() # close the file

<i><h2 align='center'>Conclusion</h2></i>  
Here we can probably summarize our results and the learners we were responsible for.  
